In [1]:
import pandas as pd
import math
import numpy as np
import jieba
import re

## non-cs (monolingual) data

In [2]:
file = open('non_cs_transcripts_h5_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [3]:
lines[-5]

['学校=644',
 '2',
 '-3.430463',
 '后面=34',
 '1',
 '-3.8931267',
 '还=87',
 '2',
 '-1.8267617',
 '一个=181',
 '2',
 '-2.7434351',
 '</s>=2',
 '2',
 '-1.2787296',
 'Total:',
 '-13.172516',
 'OOV:',
 '0']

In [4]:
len(lines)

22777

In [5]:
lines = lines[:-4]

In [6]:
lines[-1]

['学校=644',
 '2',
 '-3.430463',
 '后面=34',
 '1',
 '-3.8931267',
 '还=87',
 '2',
 '-1.8267617',
 '一个=181',
 '2',
 '-2.7434351',
 '</s>=2',
 '2',
 '-1.2787296',
 'Total:',
 '-13.172516',
 'OOV:',
 '0']

In [7]:
mean_probs = []

for line in lines:
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    vals = new_line[2::3]
    mean = 0
    for val in vals:
        mean += float(val)
    mean /= len(vals)
    mean_probs.append(mean)
        #surps.append(-float(val))

In [8]:
len(mean_probs) == len(lines)

True

In [9]:
base_10_mean_probs = mean_probs

In [10]:
df = pd.read_csv('seame_non_cs_data.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,...,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency,mean cv surprisal,random word cv surprisal
0,0,0,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,...,我们,2,2,2,8.519816876703520,24.420960,24.420960,7.519817,8.990886,8.990886
1,1,1,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,...,进,1,1,1,11.230889976662300,24.086453,24.284205,8.926648,14.903334,18.721443
2,2,2,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,...,一下,2,2,1,13.208367276887400,23.196542,30.291875,9.308506,7.458964,2.954435
3,3,3,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,...,最后,2,2,1,11.972945663039500,21.718952,24.101382,8.752136,12.588530,8.092339
4,4,4,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,...,算,1,1,1,13.708818971267000,24.420960,24.420960,12.708819,15.312369,15.312369


In [11]:
random_word = list(df['random word'])
len(random_word) == len(lines)

True

In [12]:
random_probs = []
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == random_word[i]:
            random_probs.append(float(vals[j]))
            break
            
            

In [13]:
base_10_random_probs = random_probs

In [14]:
base_10_mean_probs[0:10]

[-2.0482392,
 -3.50050315,
 -2.5301778,
 -3.0085005500000004,
 -3.6761172,
 -3.8517315,
 -4.197340725,
 -3.1363703000000003,
 -4.57329275,
 -2.9983523499999998]

In [15]:
base_10_random_probs[0:10]

[-2.0482392,
 -4.0543213,
 -2.0500326,
 -2.9462435,
 -3.6761172,
 -3.8517315,
 -3.2063828,
 -2.4304883,
 -5.9402027,
 -2.328006]

In [16]:
raw_mean_probs = [10**val for val in base_10_mean_probs]
raw_random_probs = [10**val for val in base_10_random_probs]

In [17]:
base_2_mean_probs = [math.log2(val) for val in raw_mean_probs]
base_2_random_probs = [math.log2(val) for val in raw_random_probs]

In [18]:
base_2_mean_surps = [-val for val in base_2_mean_probs]
base_2_random_surps = [-val for val in base_2_random_probs]

In [19]:
df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,...,random word,random word length,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency,mean cv surprisal,random word cv surprisal
0,0,0,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,...,我们,2,2,2,8.519816876703520,24.420960,24.420960,7.519817,8.990886,8.990886
1,1,1,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,...,进,1,1,1,11.230889976662300,24.086453,24.284205,8.926648,14.903334,18.721443
2,2,2,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,...,一下,2,2,1,13.208367276887400,23.196542,30.291875,9.308506,7.458964,2.954435
3,3,3,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,...,最后,2,2,1,11.972945663039500,21.718952,24.101382,8.752136,12.588530,8.092339
4,4,4,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,...,算,1,1,1,13.708818971267000,24.420960,24.420960,12.708819,15.312369,15.312369


In [20]:
df['mean hub5 surprisal'] = base_2_mean_surps
df['random word hub5 surprisal'] = base_2_random_surps
df.head()

,Unnamed: 0,Unnamed: 0.1,file,type,conversation,speaker,transcript_2015,filtered_transcript_2015,length_of_turn,language,...,random POS,mode POS,random word frequency,mean surprisal,random word surprisal,mean word frequency,mean cv surprisal,random word cv surprisal,mean hub5 surprisal,random word hub5 surprisal
0,0,0,01NC02FBY_0101,Conversation,1,2,我们,['我们'],1,ZH,...,2,2,8.519816876703520,24.420960,24.420960,7.519817,8.990886,8.990886,6.804103,6.804103
1,1,1,01NC02FBY_0101,Conversation,1,2,进 [啊] 没有,"['进', '没有']",2,ZH,...,1,1,11.230889976662300,24.086453,24.284205,8.926648,14.903334,18.721443,11.628420,13.468164
2,2,2,01NC02FBY_0101,Conversation,1,2,(ppl) 等一下 (ppb),"['等', '一下']",2,ZH,...,2,1,13.208367276887400,23.196542,30.291875,9.308506,7.458964,2.954435,8.405069,6.810061
3,3,3,01NC02FBY_0101,Conversation,1,2,我们 拿 多少 分 [啊] 到 最后,"['我们', '拿', '多少', '分', '到', '最后']",6,ZH,...,2,1,11.972945663039500,21.718952,24.101382,8.752136,12.588530,8.092339,9.994023,9.787209
4,4,4,01NC02FBY_0101,Conversation,1,2,算,['算'],1,ZH,...,1,1,13.708818971267000,24.420960,24.420960,12.708819,15.312369,15.312369,12.211797,12.211797


In [21]:
df.to_csv('seame_non_cs_data.csv')

## cs data (all translated to english)

In [22]:
csw_df = pd.read_csv('seame_with_eng2_zh2_translations.csv')

In [23]:
csw_df.columns

Index(['Unnamed: 0', 'file', 'type', 'conversation', 'speaker',
       'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word', 'first_cs_word_eng_pos',
       'word_length_eng', 'english_translation', 'english_wbw_translation',
       'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation',
       'cs word en surprisal', 'cs word en wbw surprisal',
       'cs word zh surprisal', 'cs word zh wbw surprisal'],
      dtype='object')

In [24]:
csw_df = csw_df[['file', 'type', 'conversation', 'speaker', 'filtered_transcript_2015', 'length_of_turn',
       'direction_of_code_switches', 'first_cs_word',
       'first_cs_word_eng_pos', 'word_length_eng', 'english_translation', 'english_wbw_translation', 'first_cs_word_zh', 'word_length_zh', 'first_cs_word_zh_pos',
       'first_cs_word_eng_raw_freq', 'first_cs_word_zh_raw_freq',
       'first_cs_word_eng_freq', 'first_cs_word_zh_freq',
       'chinese_translation', 'chinese_wbw_translation']]
csw_df.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名


In [25]:
without_todo = csw_df.replace('TODO', np.NaN)
without_todo = without_todo.dropna()
len(without_todo)

28333

In [26]:
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,english_wbw_translation,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,"uh, on october 11th, i tell you a thing to tel...",但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,october you mean finals,你,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,can't tell if we are yah,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,"i tell you one thing but can't tell us, we hav...",但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,because top four,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名


In [27]:
en_trans = list(without_todo['english_translation'])
en_trans_wbw = list(without_todo['english_wbw_translation'])

In [28]:
len(en_trans) == len(en_trans_wbw)

True

In [76]:
# file = open('en_translated_transcripts.txt','w')
# for line in en_trans:
#     file.write(line+"\n")
# file.close()

In [77]:
# file = open('en_translated_wbw_transcripts.txt','w')
# for line in en_trans_wbw:
#     file.write(line+"\n")
# file.close()

In [29]:
file = open('en_translated_transcripts_ch_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [30]:
lines = lines[:-4]

In [31]:
csw_word = list(without_todo['first_cs_word'])
len(csw_word) == len(lines)

True

In [32]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [33]:
len(base_10_probs)

28333

In [34]:
base_10_probs.count('TODO')

7450

In [35]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [36]:
without_todo['cs word en ch surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_zh,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en ch surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,但,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,你,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,6.437156
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,雅虎,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,17.303365
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,但,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,7.730491
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,顶部,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,18.334781


In [37]:
file = open('en_translated_wbw_transcripts_ch_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [38]:
lines = lines[:-4]

In [39]:
len(csw_word) == len(lines)

True

In [40]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [41]:
len(base_10_probs)

28333

In [42]:
base_10_probs.count('TODO')

274

In [43]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [44]:
without_todo['cs word en wbw ch surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,word_length_zh,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en ch surprisal,cs word en wbw ch surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,1,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,10.144894
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,1,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,6.437156,6.437156
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,2,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,17.303365,17.303365
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,1,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,7.730491,6.250333
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,2,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,18.334781,18.334781


## cs data (all translated to chinese)

In [45]:
zh_trans = list(without_todo['chinese_translation'])
zh_trans_wbw = list(without_todo['chinese_wbw_translation'])

In [46]:
len(zh_trans) == len(zh_trans_wbw)

True

In [80]:
# file = open('zh_translated_transcripts.txt','w')
# for line in zh_trans:
#     file.write(line+"\n")
# file.close()

In [81]:
# file = open('zh_translated_wbw_transcripts.txt','w')
# for line in zh_trans_wbw:
#     file.write(line+"\n")
# file.close()

In [131]:
# tokenized_zh_trans = []
# for sent in zh_trans:
#     tokenized_zh_trans.append(' '.join((jieba.lcut(sent))))

In [142]:
# file = open('zh_translated_transcripts.txt','w')
# for line in tokenized_zh_trans:
#     file.write(line+"\n")
# file.close()

In [47]:
file = open('zh_translated_transcripts_h5_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [48]:
lines = lines[:-4]

In [49]:
csw_word = list(without_todo['first_cs_word_zh'])
len(csw_word) == len(lines)

True

In [50]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [51]:
len(base_10_probs)

28333

In [52]:
base_10_probs.count('TODO')

19390

In [53]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [54]:
without_todo['cs word zh h5 surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_zh_pos,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en ch surprisal,cs word en wbw ch surprisal,cs word zh h5 surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,2,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,10.144894,TODO
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,2,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,6.437156,6.437156,6.801896
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,0,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,17.303365,17.303365,TODO
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,2,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,7.730491,6.250333,TODO
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,0,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,18.334781,18.334781,TODO


In [55]:
file = open('zh_translated_wbw_transcripts_h5_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [56]:
lines = lines[:-4]

In [57]:
len(csw_word) == len(lines)

True

In [58]:
base_10_probs = ['TODO'] * len(lines)
for i in range(len(lines)):
    line = lines[i]
    end_index = line.index('</s>=2')
    new_line = line[:end_index]
    words = new_line[0::3]
    vals = new_line[2::3]
    clean_words = []
    for word in words:
        end_index = word.index('=')
        word = word[:end_index]
        clean_words.append(word)
    for j in range(len(clean_words)):
        word = clean_words[j]
        if word == csw_word[i]:
            base_10_probs[i] = float(vals[j])
            break           

In [59]:
len(base_10_probs)

28333

In [60]:
base_10_probs.count('TODO')

10662

In [61]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [62]:
without_todo['cs word zh wbw h5 surprisal'] = base_2_surps
without_todo.head()

,file,type,conversation,speaker,filtered_transcript_2015,length_of_turn,direction_of_code_switches,first_cs_word,first_cs_word_eng_pos,word_length_eng,...,first_cs_word_eng_raw_freq,first_cs_word_zh_raw_freq,first_cs_word_eng_freq,first_cs_word_zh_freq,chinese_translation,chinese_wbw_translation,cs word en ch surprisal,cs word en wbw ch surprisal,cs word zh h5 surprisal,cs word zh wbw h5 surprisal
0,01NC01FBX_0101,Conversation,1,1,"['呃', '十一号', '十月', '哦', '我', '跟', '你', '讲', '一...",15,"['man > eng', 'eng > man']",but,2,3,...,791724336,48791767,8.812799,9.104577,呃十一十月我跟讲讲一一一个一个个不出去,呃 十一号 十月 哦 我 跟 你 讲 一个 东西 但 不 可以 讲出去,TODO,10.144894,TODO,11.729906
1,01NC02FBY_0101,Conversation,1,2,"['十月', 'you', 'mean', 'finals']",4,['man > eng'],you,2,3,...,950006304,25465651,8.549860,10.042662,十月你是说决赛,十月 你 的 意思 是 决赛,6.437156,6.437156,6.801896,6.801896
2,01NC02FBY_0101,Conversation,1,2,"['不', '可以', '讲出去', '我们', '是不是', 'yah']",6,['man > eng'],yah,0,3,...,131496,11734,21.368562,21.126304,不可以讲我们不是,不 可以 讲出去 我们 是不是 雅虎,17.303365,17.303365,TODO,17.237802
3,01NC01FBX_0101,Conversation,1,1,"['我', '跟', '你', '讲', '一个', '事', 'but', '不', '可...",30,"['man > eng', 'eng > man', 'man > eng', 'eng >...",but,2,3,...,791724336,48791767,8.812799,9.104577,我我讲一一一个个一一一可以讲出去有有有有有进有有有有进我们我们我们我们第三第三拿拿拿拿拿拿拿...,我 跟 你 讲 一个 事 但 不 可以 讲出去 我们 有进 然后 我们 拿 第三 但是 先生...,7.730491,6.250333,TODO,7.609494
4,01NC01FBX_0101,Conversation,1,1,"['因为', 'top', 'four']",3,['man > eng'],top,2,3,...,40791068,0,13.091472,34.644711,因为前四名,因为 前 四名,18.334781,18.334781,TODO,TODO


In [63]:
without_todo_dropped = without_todo.replace('TODO', np.NaN)
without_todo_dropped = without_todo_dropped.dropna()
len(without_todo_dropped)

6171

In [64]:
without_todo.to_csv('seame_with_eng2_zh2_translations_fixed_surps.csv')
without_todo_dropped.to_csv('seame_with_eng2_zh2_translations_no_todos_fixed_surps.csv')

## hub5 and callhome monolingual data

In [65]:
file = open('callhome_en_sample_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [66]:
lines = lines[:-4]

In [67]:
base_10_probs = []
for i in range(len(lines)):
    line = lines[i]
    try:
        end_index = line.index('</s>=2')
        new_line = line[:end_index]
        #words = new_line[0::3]
        vals = new_line[2::3]
        float_vals = [float(val) for val in vals]
        base_10_probs.extend(float_vals)
    except:
        new_line = line
        vals = new_line[2::3]
        float_vals = [float(val) for val in vals]
        base_10_probs.extend(float_vals)
            

In [68]:
len(base_10_probs)

38393

In [69]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [70]:
file = open('callhome_en_sample_surprisals.txt','w')
for line in base_2_surps:
    file.write(str(line)+"\n")
file.close()

In [71]:
np.mean(base_2_surps) #mean surprisal of callhome english

7.678528503882545

In [72]:
file = open('hub5_sample_output.txt', 'r')
lines = []
for line in file.readlines(): 
    lines.append(line.split())

In [73]:
lines[-5]

['我=15',
 '2',
 '-1.3018283',
 '</s>=2',
 '3',
 '-1.5886003',
 'Total:',
 '-2.8904285',
 'OOV:',
 '0']

In [74]:
lines = lines[:-4]

In [75]:
base_10_probs = []
for i in range(len(lines)):
    line = lines[i]
    try:
        end_index = line.index('</s>=2')
        new_line = line[:end_index]
        #words = new_line[0::3]
        vals = new_line[2::3]
        float_vals = [float(val) for val in vals]
        base_10_probs.extend(float_vals)
    except:
#         new_line = line
#         vals = new_line[2::3]
#         float_vals = [float(val) for val in vals]
#         base_10_probs.extend(float_vals)
        print(i)
        pass
            

In [76]:
len(base_10_probs)

42482

In [77]:
raw_probs = [10**val if type(val) is float else val for val in base_10_probs]
base_2_probs = [math.log2(val) if type(val) is float else val for val in raw_probs]
base_2_surps = [-val if type(val) is float else val for val in base_2_probs]

In [78]:
file = open('hub5_sample_surprisals.txt','w')
for line in base_2_surps:
    file.write(str(line)+"\n")
file.close()

In [79]:
np.mean(base_2_surps) #mean surprisal of hub5 mandarin

8.794762814281492